# Mask RCNN Training - Watermelon

Use this notebook to train the model in Google Colabs.
Save the history and model weights for use in local evaluation and prediciton. 

This script was adapted from the [Matterport Balloons Example](https://github.com/matterport/Mask_RCNN/tree/master/samples/balloon)

Images were annotated using the online VGG Image Annotator [(VIA)](http://www.robots.ox.ac.uk/~vgg/software/via/via.html)

## Load Mask RCNN into Google Colab

In [2]:
!git clone https://github.com/matterport/Mask_RCNN

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.84 MiB | 29.85 MiB/s, done.
Resolving deltas: 100% (569/569), done.


## Set up the GPU Environment
In Colabs go to:  
Edit > Notebook Settings  
Set the hardware accelerator to 'GPU' and reload the notebook. 

In [3]:
#RUN GPU
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


'\n#RUN TPU\n%tensorflow_version 1.x\nimport tensorflow as tf\nprint("Tensorflow version " + tf.__version__)\ntry:\n  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection\n  print(\'Running on TPU \', tpu.cluster_spec().as_dict()[\'worker\'])\nexcept ValueError:\n  raise BaseException(\'ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!\')\n'

## Move into the Mask RCNN directory and install the package

In [0]:
import os
os.chdir('Mask_RCNN')

In [5]:
!ls

assets	LICENSE      mrcnn	requirements.txt  setup.cfg
images	MANIFEST.in  README.md	samples		  setup.py


In [6]:
!python3 setup.py install;

running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/config.py -> build/lib/mrcnn
copying mrcnn/__init__.py -> build/lib/mrcnn
copying mrcnn/visualize.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/lib/mrcnn
copying mrcnn/utils.py -> build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/config.py -> build/bdist.linux-x86_64/egg/mrcn

## Load Modules

In [7]:
import mrcnn
import mrcnn.model as modellib
from mrcnn.config import Config
from mrcnn import utils
from mrcnn.model import MaskRCNN

import numpy as np
import colorsys
import argparse
import random
import os
import sys
import time
import json
import skimage
import datetime
from keras.models import load_model


Using TensorFlow backend.


## Link Google Drive and set up directories

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
homedir='/content/drive/My Drive/Colab Data/'
datadir=homedir+'/RCNN_test/'

In [10]:
#files need to be accessed using os !ls does not work.
os.listdir(homedir)

['RCNN_test', 'mask_rcnn_coco.h5', 'watermelon_annot.json', 'saved']

## Define the Config and Dataset classes

In [11]:
class myMaskRCNNConfig(Config):
    NAME = "MaskRCNN_config"
 
    # set the number of GPUs to use along with the number of images
    # per GPU
    # Colabs provides a 12GB GPU which should fit 2 images
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
 
    # number of classes (we would normally add +1 for the background)
    # watermelon + BG
    NUM_CLASSES = 1+1
   
    # Number of training steps per epoch (set to number of training samples)
    STEPS_PER_EPOCH = 80
    
    # Learning rate (reduce to stop NaN losses)
    LEARNING_RATE=0.001
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    # setting Max ground truth instances
    MAX_GT_INSTANCES=10
    
config = myMaskRCNNConfig()
config.display()

check=[]
class WatermelonDataset(utils.Dataset):

    def load_dataset(self, dataset_dir):
        """
        Load the images and annotations from Google Drive
        The mask coordinates are save in 'polygons'
        """

        self.add_class("watermelon", 1, "watermelon")

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations = json.load(open(os.path.join(homedir, "watermelon_annot.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        for a in annotations:
            if a['filename'] in os.listdir(dataset_dir):
                # Get the x, y coordinates of points of the polygons that make up
                # the outline of each object instance. These are stored in the
                # shape_attributes (see json format above)
        
                if type(a['regions']) is dict:
                        polygons = [r['shape_attributes'] for r in a['regions'].values()]
                else:
                        polygons = [r['shape_attributes'] for r in a['regions']]
        
                # load_mask() needs the image size to convert polygons to masks.
                # Unfortunately, VIA doesn't include it in JSON, so we must read
                # the image. This is only managable since the dataset is tiny.
                image_path = os.path.join(dataset_dir, a['filename'])
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]
    
                self.add_image(
                    "watermelon",
                    image_id=a['filename'],
                    path=image_path,
                    width=width, height=height,
                    polygons=polygons)

    def load_mask(self, image_id):
        """
        Generate instance masks for an image.
        
        Returns:
        
        masks: A bool array of shape [height, width, instance count] with
               one mask per instance.
        
        class_ids: a 1D array of class IDs of the instance masks.
        
        """
        # If not a melon dataset image, delegate to background class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "watermelon":
            print('not a melon!')
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)

        class_id=[]
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1
            class_id.append(self.class_names.index('watermelon'))

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask, np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "watermelon":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

# Prepare the Training and Test Sets

In [12]:
# prepare train set
train_set = WatermelonDataset()
train_set.load_dataset(datadir+'/train')
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# prepare test/val set
test_set = WatermelonDataset()
test_set.load_dataset(datadir+'/test')
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Train: 80
Test: 20


# Prepare the Training Model:

In [14]:
print("Loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="training", config=config, model_dir='./')

#load the weights for COCO
model.load_weights(homedir+'/mask_rcnn_coco.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

Loading Mask R-CNN model...






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead








## Train the Model:
If you begin to encounter NaNs in your loss values, try lowering the learning rate by 4 or so times. 

In [15]:
# train heads with higher lr to speedup the learning
model.train(train_set, test_set, learning_rate=2*config.LEARNING_RATE, epochs=15, layers='heads')
history = model.keras_model.history.history


Starting at epoch 0. LR=0.002

Checkpoint Path: ./maskrcnn_config20200218T1359/mask_rcnn_maskrcnn_config_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4   

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/15
80/80 [==============================] - 172s 2s/step - loss: 1.4619 - rpn_class_loss: 0.0121 - rpn_bbox_loss: 0.3595 - mrcnn_class_loss: 0.1061 - mrcnn_bbox_loss: 0.7476 - mrcnn_mask_loss: 0.2366 - val_loss: 0.8698 - val_rpn_class_loss: 0.0042 - val_rpn_bbox_loss: 0.2365 - val_mrcnn_class_loss: 0.0299 - val_mrcnn_bbox_loss: 0.4140 - val_mrcnn_mask_loss: 0.1852

Epoch 2/15
80/80 [==============================] - 119s 1s/step - loss: 0.7857 - rpn_class_loss: 0.0062 - rpn_bbox_loss: 0.2807 - mrcnn_class_loss: 0.0584 - mrcnn_bbox_loss: 0.2413 - mrcnn_mask_loss: 0.1991 - val_loss: 0.6193 - val_rpn_class_loss: 0.0036 - val_rpn_bbox_loss: 0.2809 - val_mrcnn_class_loss: 0.0136 - val_mrcnn_bbox_loss: 0.1486 - val_mrcnn_mask_loss: 0.1727
Epoch 3/15
80/80 [==============================] - 121s 2s/step - loss: 0.5686 - rpn_class_loss: 0.0048 - rpn_bbox_loss: 0.2151 - mrcnn_class_loss: 0.0400 - mrcnn_bbox_loss: 0.1364 - mrcnn_mask_loss: 0.1724 - val_loss: 0.6253 - val_rpn_class_loss

## Check the Output
Find the output directory name.  
In this example it is: 'maskrcnn_config20200218T1359'

In [16]:
!ls

assets	images	     maskrcnn_config20200218T1359  README.md	     setup.cfg
build	LICENSE      mask_rcnn.egg-info		   requirements.txt  setup.py
dist	MANIFEST.in  mrcnn			   samples


In [0]:
#Move to the latest model directory
os.chdir('./maskrcnn_config20200218T1359')

## Save Model Weights and History in Google Drive:
Find the name of the latest model weights to save

In [19]:
!ls -lhrt

total 3.7G
-rw-r--r-- 1 root root 245M Feb 18 14:03 mask_rcnn_maskrcnn_config_0001.h5
-rw-r--r-- 1 root root 245M Feb 18 14:05 mask_rcnn_maskrcnn_config_0002.h5
-rw-r--r-- 1 root root 245M Feb 18 14:07 mask_rcnn_maskrcnn_config_0003.h5
-rw-r--r-- 1 root root 245M Feb 18 14:09 mask_rcnn_maskrcnn_config_0004.h5
-rw-r--r-- 1 root root 245M Feb 18 14:11 mask_rcnn_maskrcnn_config_0005.h5
-rw-r--r-- 1 root root 245M Feb 18 14:14 mask_rcnn_maskrcnn_config_0006.h5
-rw-r--r-- 1 root root 245M Feb 18 14:16 mask_rcnn_maskrcnn_config_0007.h5
-rw-r--r-- 1 root root 245M Feb 18 14:18 mask_rcnn_maskrcnn_config_0008.h5
-rw-r--r-- 1 root root 245M Feb 18 14:20 mask_rcnn_maskrcnn_config_0009.h5
-rw-r--r-- 1 root root 245M Feb 18 14:22 mask_rcnn_maskrcnn_config_0010.h5
-rw-r--r-- 1 root root 245M Feb 18 14:24 mask_rcnn_maskrcnn_config_0011.h5
-rw-r--r-- 1 root root 245M Feb 18 14:26 mask_rcnn_maskrcnn_config_0012.h5
-rw-r--r-- 1 root root 245M Feb 18 14:28 mask_rcnn_maskrcnn_config_0013.h5
-rw-r--r-- 1 r

In [20]:
now_str='20200218T1359'

try:
  os.makedirs(homedir+'saved')
except:
  print('Save directory exists!')

weights_fname =homedir+'saved/weights_watermelon_'+now_str+'.h5'
history_fname =homedir+'saved/history_watermelon_'+now_str+'.json'

model.keras_model.save_weights(weights_fname)

with open(history_fname,"w") as f:
   f.write(json.dumps(history))

Save directory exists!
